In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

df = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')
display(df.head(3))

產品集合 = set(df['品名'].values)

train_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)
train_df = train_df.dropna(axis=0)
display(train_df.head(3))

val_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)
val_df = val_df.dropna(axis=0)
display(val_df.head(3))

,RIGID,分機,公司代號,公司事業部門,品名,新增
0,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID FLAKE PEARLS MICROPEARLS,NaN
1,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID,NaN
2,黃淑玲,7190,11,台塑塑膠部,COMMODITY FORMOSACN,NaN


,string_X_train,string_Y_1,string_Y_2,Y_label,row_id
586,"YUNGSOX 2100M 12MT USD1,015/MT USD12,180 YUNGS...",0,13,YUNGSOX 2100M,855
92,"TRIS 1,2-CYCLOHEXANE DICARBOXYLIC ACID,DI-ISON...",0,50,"TRIS 1,2-CYCLOHEXANE DICARBOXYLIC ACID,DI-ISON...",140
91,"TRIS . CIF PORT KLANG,MALAYSIA",0,4,TRIS,139


,string_X_train,string_Y_1,string_Y_2,Y_label,row_id
1,"COMMODITY: STYRENE MONOMER IN BULKQUANTITY: 3,...",11,34,STYRENE MONOMER IN BULK,1
2,"COMMODITY: STYRENE MONOMER IN BULKQUANTITY: 3,...",11,34,STYRENE MONOMER IN BULK,2
3,PP 3307UNC1 . TRADE TERMS: CFR ANY JAPANESE PORT,0,2,PP,10


# find_fail_sample and drop fail_sample

In [2]:
def find_fail_sample(df):
    fails = []
    for i in df.index:
        context = df.loc[i,'string_X_train']
        answer = df.loc[i,'Y_label']
        if answer not in context:
            fails.append(i)
    return fails
train_fails = find_fail_sample(train_df)
val_fails = find_fail_sample(val_df)
print(train_fails,val_fails)
display(val_df.loc[val_fails])
print(val_df.shape)
val_df = val_df.drop(val_fails,axis=0)
print(val_df.shape)

[] [342, 343, 344]


,string_X_train,string_Y_1,string_Y_2,Y_label,row_id
342,#NAME?,1,26,PURIFIED ISOPHTHALIC ACID,1238
343,#NAME?,1,26,PURIFIED ISOPHTHALIC ACID,1240
344,#NAME?,40,65,PURIFIED ISOPHTHALIC ACID,1241


(744, 5)
(741, 5)


In [3]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p)
        labels[i] = products
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/741 [00:00<?, ?it/s]

In [4]:
result

,string_X_train,string_Y_1,string_Y_2,Y_label,row_id,predict
1,"COMMODITY: STYRENE MONOMER IN BULKQUANTITY: 3,...",11,34,STYRENE MONOMER IN BULK,1,"[STYRENE MONOMER, MA]"
2,"COMMODITY: STYRENE MONOMER IN BULKQUANTITY: 3,...",11,34,STYRENE MONOMER IN BULK,2,"[STYRENE MONOMER, MA, PE]"
3,PP 3307UNC1 . TRADE TERMS: CFR ANY JAPANESE PORT,0,2,PP,10,"[PP, PA]"
4,"CIF BELAWAN PORT, INDONESIA +20 MT +/- 5PCT OF...",47,62,2-ETHYL HEXANOL,13,[2-ETHYL HEXANOL]
5,"CIF BELAWAN PORT, INDONESIA +20 MTON +/-5PCT O...",48,63,2-ETHYL HEXANOL,14,[2-ETHYL HEXANOL]
...,...,...,...,...,...,...
740,3 FCLS -43.20 MTS OF ETHYL ACRYLATE AT THE RAT...,21,35,ETHYL ACRYLATE,2514,"[PA, EVA, ACRYLATE, PE, ETHYL ACRYLATE]"
741,METHYL METHACRYLATE 99.98 PCT MIN QTY 1 FCL EQ...,0,19,METHYL METHACRYLATE,2518,"[MA, EVA, METHYL METHACRYLATE, ACRYLATE, PE]"
742,PHTHALIC ANHYRIDE PA QTY 180 MTS AS PER PROFOR...,18,20,PA,2524,"[MA, PA, EVA, PE]"
743,TRADE TERM: CIF HONG KONG DESCRIPTIONS QUANTIT...,76,104,ETHYLENE-PROPYLENE COPOLYMER,2525,"[INA , MA, POLYPROPYLENE, ETHYLENE, MATERIAL, ..."


In [5]:
def get_acc(df):
    correct = []
    correct_label = []
    for i in df.index:
        if df.loc[i,'Y_label'] in df.loc[i,'predict']:
            correct.append('yes')
        else:
            correct.append('no')
    return correct

In [6]:
correct = get_acc(result)
result['correct'] = correct
result

,string_X_train,string_Y_1,string_Y_2,Y_label,row_id,predict,correct
1,"COMMODITY: STYRENE MONOMER IN BULKQUANTITY: 3,...",11,34,STYRENE MONOMER IN BULK,1,"[STYRENE MONOMER, MA]",no
2,"COMMODITY: STYRENE MONOMER IN BULKQUANTITY: 3,...",11,34,STYRENE MONOMER IN BULK,2,"[STYRENE MONOMER, MA, PE]",no
3,PP 3307UNC1 . TRADE TERMS: CFR ANY JAPANESE PORT,0,2,PP,10,"[PP, PA]",yes
4,"CIF BELAWAN PORT, INDONESIA +20 MT +/- 5PCT OF...",47,62,2-ETHYL HEXANOL,13,[2-ETHYL HEXANOL],yes
5,"CIF BELAWAN PORT, INDONESIA +20 MTON +/-5PCT O...",48,63,2-ETHYL HEXANOL,14,[2-ETHYL HEXANOL],yes
...,...,...,...,...,...,...,...
740,3 FCLS -43.20 MTS OF ETHYL ACRYLATE AT THE RAT...,21,35,ETHYL ACRYLATE,2514,"[PA, EVA, ACRYLATE, PE, ETHYL ACRYLATE]",yes
741,METHYL METHACRYLATE 99.98 PCT MIN QTY 1 FCL EQ...,0,19,METHYL METHACRYLATE,2518,"[MA, EVA, METHYL METHACRYLATE, ACRYLATE, PE]",yes
742,PHTHALIC ANHYRIDE PA QTY 180 MTS AS PER PROFOR...,18,20,PA,2524,"[MA, PA, EVA, PE]",yes
743,TRADE TERM: CIF HONG KONG DESCRIPTIONS QUANTIT...,76,104,ETHYLENE-PROPYLENE COPOLYMER,2525,"[INA , MA, POLYPROPYLENE, ETHYLENE, MATERIAL, ...",yes


In [7]:
result['correct'].value_counts()

yes    389
no     352
Name: correct, dtype: int64

In [8]:
result['correct'].value_counts()['yes']/len(val_df)

0.524966261808367